In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%autosave 20

Autosaving every 20 seconds


In [2]:
from model import Net

In [3]:
import torch
from torchsummary import summary
from optimizer import *
from data_loader import *
from run import *
import torch
from model import Net
from optimizer import get_optimizer,run_lrfinder
from model_fit import training,testing
import albumentations as A
from albumentations.augmentations.geometric.resize import Resize
from albumentations.pytorch.transforms import ToTensorV2
from data_loader import MNISTDataLoader
import json
from torch import nn
import plotext as plt
import torch
from torch.optim import SGD
from torch.optim.lr_scheduler import StepLR
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR,ReduceLROnPlateau
import numpy as np
from torch_lr_finder import LRFinder
import os
import sys

# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))


In [4]:
config = load_config()

In [5]:
config['training']['epochs'] = 15

In [6]:
config['training']['epochs']

15

In [7]:
def set_seed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    # If using torch.backends.cudnn, set the following for reproducibility
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def get_model(device):
    model = Net().to(device)
    # model = model.float()
    return model


def run_model(model,device,batch_size,epochs,optimizer,scheduler,use_scheduler,best_model):
    train_losses = []
    train_accuracy = []
    test_losses =[]
    test_accuracy = []
    # print(scheduler)
    # print(optimizer)
    # summary(model, (1,28, 28 )).to(device)
    _ = receptive_field(model,28)

    for EPOCHS in range(0,epochs):
        train_loss, train_acc = training(model,device,train_loader,optimizer,EPOCHS)
        train_losses.append(train_loss)
        train_accuracy.append(train_acc)

        test_loss,test_acc = testing(model,device,test_loader,EPOCHS)
        test_accuracy.append(test_acc)
        test_losses.append(test_loss)
        
        if (scheduler_type == 'reducelronplateau') & (use_scheduler ==True):
            scheduler.step(test_loss)
            current_lr = optimizer.param_groups[0]['lr']
        elif (scheduler_type == 'steplr') & (use_scheduler ==True):
            scheduler.step()
            current_lr = scheduler.get_last_lr()[0]
        else:
            current_lr = optimizer.param_groups[0]['lr']

        try:
            if len(test_accuracy) > 1:
                if (EPOCHS >= 3 and 
                    max(test_accuracy[:-1]) < test_accuracy[-1] and 
                    max(test_accuracy) >= best_model):
                    
                    checkpoint = {
                        'epoch': EPOCHS + 1,
                        'valid_loss_min': test_losses[-1],
                        'state_dict': model.state_dict(),
                        'optimizer': optimizer.state_dict(),
                    }
                    
                    file_name = f"./model_folder/modelbest_{test_accuracy[-1]:.4f}_epoch_{EPOCHS}.pt"
                    torch.save(checkpoint, file_name)
                    print(f"Target Achieved: {max(test_accuracy) * 100:.2f}% Test Accuracy!!")
                else:
                    print("Conditions not met for saving the model.")
            else:
                print("Insufficient test accuracy data.")
        except Exception as e:
            print(f"Model saving failed: {e}")

        print(f"LR: {current_lr}\n")
    return model,train_losses, train_accuracy,test_losses,test_accuracy


def load_config():
    with open('config.json', 'r') as f:
        return json.load(f)


def get_loss_function(loss_type):
    if loss_type is None:
        return nn.NLLLoss()   
    loss_types = {
        'cross_entropy': nn.CrossEntropyLoss(),
        'mse': nn.MSELoss(),
        'nll': nn.NLLLoss()
    }
    return loss_types.get(loss_type.lower(), nn.CrossEntropyLoss())

In [8]:
config = load_config()
# Set seed from config
set_seed(config['seed'])

In [9]:
# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Get loss function and scheduler settings from config
loss_fn = get_loss_function(config['training'].get('loss_type'))
use_scheduler = config['training'].get('use_scheduler', False)
scheduler_type = config['training'].get('scheduler_type', 'steplr')
runlr_finer = config['training'].get('runlr_finer', False)
use_scheduler = bool(use_scheduler)
runlr_finer = bool(runlr_finer)

best_model = config['best_model']

In [10]:
device
loss_fn
use_scheduler
scheduler_type
runlr_finer
use_scheduler
runlr_finer
best_model

'cuda'

NLLLoss()

False

'reducelronplateau'

False

False

False

0.991

In [11]:
# Set seed from config
_ = torch.manual_seed(config['seed'])
if torch.cuda.is_available():
    _ = torch.cuda.manual_seed(config['seed'])

In [12]:
# Get batch size based on device
batch_size = config['training']['batch_size']
epochs = config['training']['epochs']

In [13]:
batch_size
epochs

128

3

In [14]:
model = get_model(device)
data_loader = MNISTDataLoader(batch_size=batch_size)
train_loader, test_loader = data_loader.get_data_loaders()

Mean: 0.1307 Std: 0.3081


In [17]:
train_loader.dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: .data
    Split: Train
    StandardTransform
Transform: Compose(
               Resize(size=(28, 28), interpolation=bilinear, max_size=None, antialias=True)
               ColorJitter(brightness=(0.9, 1.1), contrast=(0.9, 1.1), saturation=(0.9, 1.1), hue=(-0.1, 0.1))
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

In [18]:
test_loader.dataset

Dataset MNIST
    Number of datapoints: 10000
    Root location: .data
    Split: Test
    StandardTransform
Transform: Compose(
               Resize(size=(28, 28), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

In [25]:
if runlr_finer:
    lrs,_ = run_lrfinder(
        model, 
        device, 
        train_loader, 
        test_loader, 
        start_lr=config['training']['start_lr'],
        end_lr=config['training']['end_lr'],
            lr_iter=config['training'].get('lr_iter', 1000)
        )
    print(lrs)
else:
    lrs = [0.015]

In [26]:
optimizer,scheduler = get_optimizer(model,scheduler = use_scheduler,\
                          scheduler_type = scheduler_type,lr = lrs[0])

In [27]:
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.015
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)

In [23]:
summary(model,(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]              90
       BatchNorm2d-2           [-1, 10, 26, 26]              20
              ReLU-3           [-1, 10, 26, 26]               0
           Dropout-4           [-1, 10, 26, 26]               0
            Conv2d-5           [-1, 10, 24, 24]             900
       BatchNorm2d-6           [-1, 10, 24, 24]              20
              ReLU-7           [-1, 10, 24, 24]               0
           Dropout-8           [-1, 10, 24, 24]               0
            Conv2d-9           [-1, 15, 22, 22]           1,350
      BatchNorm2d-10           [-1, 15, 22, 22]              30
             ReLU-11           [-1, 15, 22, 22]               0
          Dropout-12           [-1, 15, 22, 22]               0
        MaxPool2d-13           [-1, 15, 11, 11]               0
           Conv2d-14           [-1, 10,

In [24]:
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)

In [ ]:
model,train_losses, train_accuracy,test_losses,test_accuracy= run_model(model,device,batch_size,epochs,optimizer,scheduler,use_scheduler,best_model)

In [ ]:
print("Max Train Accuracy: ",max(train_accuracy))
print("Max Test Accuracy: ",max(test_accuracy))

=======================================Reciptive Field Calculator========================================
|    | Kernel_size   | Padding   |   Stride | Input_Img_size   | Output_Img_size   | Receptive_field   |
|---:|:--------------|:----------|---------:|:-----------------|:------------------|:------------------|
|  0 | 3*3           | NO        |        1 | 28*28            | 26*26             | 3*3               |
|  1 | 3*3           | NO        |        1 | 26*26            | 24*24             | 5*5               |
|  2 | 3*3           | NO        |        1 | 24*24            | 22*22             | 7*7               |
|  3 | 2*2           | NO        |        2 | 22*22            | 11*11             | 8*8               |
|  4 | 1*1           | NO        |        1 | 11*11            | 11*11             | 8*8               |
|  5 | 3*3           | NO        |        1 | 11*11            | 9*9               | 12*12             |
|  6 | 3*3           | NO        |        1 | 9*9     

Train ==> Epochs: 0 Batch:  468 loss: 0.3084717094898224 Accuracy: 78.57% : 100%|█| 469/469 [0
Test ==> Epochs: 0 Batch:  78 loss: 0.23437826504707338 Accuracy: 94.59% : 100%|█| 79/79 [00:0


Insufficient test accuracy data.
LR: 0.001



Train ==> Epochs: 1 Batch:  468 loss: 0.1329829841852188 Accuracy: 94.90% : 100%|█| 469/469 [0
Test ==> Epochs: 1 Batch:  78 loss: 0.12610795253515245 Accuracy: 96.80% : 100%|█| 79/79 [00:0


Conditions not met for saving the model.
LR: 0.001



...

Conditions not met for saving the model.
LR: 0.001



Train ==> Epochs: 13 Batch:  468 loss: 0.04736688733100891 Accuracy: 98.75% : 100%|█| 469/469 
Test ==> Epochs: 13 Batch:  78 loss: 0.039087547039985654 Accuracy: 98.84% : 100%|█| 79/79 [00


Conditions not met for saving the model.
LR: 0.001



Train ==> Epochs: 14 Batch:  460 loss: 0.032683372497558594 Accuracy: 98.82% :  98%|▉| 461/469


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
# Create a figure with 1 row and 2 columns of subplots
fig, axes = plt.subplots(1, 2, figsize=(12, 5))  # Adjust figsize as needed

# Plot accuracy on the first subplot
axes[0].plot(train_accuracy, label="Train Accuracy")
axes[0].plot(test_accuracy, label="Test Accuracy")
axes[0].set_title("Model Accuracy")
axes[0].set_xlabel("Epochs")
axes[0].set_ylabel("Accuracy")
axes[0].legend()

# Plot loss on the second subplot
axes[1].plot(train_losses, label="Train Loss")
axes[1].plot(test_losses, label="Test Loss")
axes[1].set_title("Model Loss")
axes[1].set_xlabel("Epochs")
axes[1].set_ylabel("Loss")
axes[1].legend()

# Adjust layout and display the plots
plt.tight_layout()
plt.show()

In [ ]:
!pytest -v

In [ ]:
!ls -R .github/